# Calculating Z-Scores in Pandas


Recall that the Z-score tells you the precise numerical value of the standard deviation for an individual data point in your sample.

When you know the Z-score for two things that are measured in different ways, we know their "standard" scores which allows us to compare one against the other!


In [1]:
## import our usual
import pandas as pd

## another package with mad math functions
from scipy.stats import zscore 


# Create a sample df
df = pd.DataFrame({'numbers': [1,2,3,4,5,6,7,8,9,3,4,6,5,7,3,2,9]})
df


,numbers
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,3


In [2]:
## get stats
df.describe()

,numbers
count,17.000000
mean,4.941176
std,2.486729
min,1.000000
25%,3.000000
50%,5.000000
75%,7.000000
max,9.000000


In [3]:
df["numbers"].mean()

4.9411764705882355

In [4]:
df["numbers"].std()

2.4867294845663785

In [7]:
df["numbers"]

0     1
1     2
2     3
3     4
4     5
5     6
6     7
7     8
8     9
9     3
10    4
11    6
12    5
13    7
14    3
15    2
16    9
Name: numbers, dtype: int64

In [6]:
# get rounded stats
round(df.describe(),2)

,numbers
count,17.00
mean,4.94
std,2.49
min,1.00
25%,3.00
50%,5.00
75%,7.00
max,9.00


In [8]:
# Calculate the zscores and drop zscores into new column
df["zscore"] = zscore(df["numbers"])
df

,numbers,zscore
0,1,-1.633660
1,2,-1.219150
2,3,-0.804639
3,4,-0.390128
4,5,0.024383
5,6,0.438894
6,7,0.853405
7,8,1.267916
8,9,1.682426
9,3,-0.804639


## Climate Comparison

You're a climate reporter and you're comparing temperatures in the North and South poles. Which region had the most usually warm day?


In [9]:
## import this data
df = pd.read_csv("https://raw.githubusercontent.com/sandeepmj/datasets/main/zscore-climate.csv")
df

,Date,Pole,Temp
0,1/29/23,Antarctica,-55
1,1/29/23,Arctic,-17
2,1/30/23,Antarctica,-68
3,1/30/23,Arctic,-24
4,1/31/23,Antarctica,-82
5,1/31/23,Arctic,-32
6,2/1/23,Antarctica,-76
7,2/1/23,Arctic,-22


## Method 1 - two dataframes

In [12]:
## get arctic data into its own df
df_n = df.query("Pole == 'Arctic'").copy()
df_n

,Date,Pole,Temp
1,1/29/23,Arctic,-17
3,1/30/23,Arctic,-24
5,1/31/23,Arctic,-32
7,2/1/23,Arctic,-22


In [13]:
## get Antarctica data into its own df
df_s = df.query("Pole == 'Antarctica'").copy()
df_s

,Date,Pole,Temp
0,1/29/23,Antarctica,-55
2,1/30/23,Antarctica,-68
4,1/31/23,Antarctica,-82
6,2/1/23,Antarctica,-76


In [14]:
## get zscore for Antarctica
df_s["zscore"] = zscore(df_s["Temp"])
df_s

,Date,Pole,Temp,zscore
0,1/29/23,Antarctica,-55,1.508589
2,1/30/23,Antarctica,-68,0.222579
4,1/31/23,Antarctica,-82,-1.162356
6,2/1/23,Antarctica,-76,-0.568812


In [15]:
## get zscore for Arctic
df_n["zscore"] = zscore(df_n["Temp"])
df_n

,Date,Pole,Temp,zscore
1,1/29/23,Arctic,-17,1.249411
3,1/30/23,Arctic,-24,-0.046274
5,1/31/23,Arctic,-32,-1.527058
7,2/1/23,Arctic,-22,0.323921


In [16]:
df.groupby("Pole")[["Temp"]].mean()

,Temp
Pole,
Antarctica,-70.25
Arctic,-23.75


In [18]:
df.groupby("Pole")[["Temp"]].std()

,Temp
Pole,
Antarctica,11.672618
Arctic,6.238322


#### Which day was more unusually warmer?

## Method 2 - Pivot

In [19]:
## recall our df
df

,Date,Pole,Temp
0,1/29/23,Antarctica,-55
1,1/29/23,Arctic,-17
2,1/30/23,Antarctica,-68
3,1/30/23,Arctic,-24
4,1/31/23,Antarctica,-82
5,1/31/23,Arctic,-32
6,2/1/23,Antarctica,-76
7,2/1/23,Arctic,-22


```df.pivot(columns = "columns you want to pivot",
index = "What your new index should be",
values = "What values are for your columns"```

In [21]:
## pivot df
dfp = df.pivot(columns = "Pole",
        index = "Date",
        values = "Temp")

dfp

Pole,Antarctica,Arctic
Date,,
1/29/23,-55,-17
1/30/23,-68,-24
1/31/23,-82,-32
2/1/23,-76,-22


In [23]:
## call the Antarctica series
dfp["Antarctica"]

Date
1/29/23   -55
1/30/23   -68
1/31/23   -82
2/1/23    -76
Name: Antarctica, dtype: int64

In [24]:
## create z-score for Antarctica
dfp["Antarctica-zscore"] = zscore(dfp["Antarctica"])
dfp

Pole,Antarctica,Arctic,Antarctica-zscore
Date,,,
1/29/23,-55,-17,1.508589
1/30/23,-68,-24,0.222579
1/31/23,-82,-32,-1.162356
2/1/23,-76,-22,-0.568812


In [25]:
## create z-score for Arctic
dfp["Arctic-zscore"] = zscore(dfp["Arctic"])
dfp

Pole,Antarctica,Arctic,Antarctica-zscore,Arctic-zscore
Date,,,,
1/29/23,-55,-17,1.508589,1.249411
1/30/23,-68,-24,0.222579,-0.046274
1/31/23,-82,-32,-1.162356,-1.527058
2/1/23,-76,-22,-0.568812,0.323921


In [27]:
dfp.sort_values(by="Antarctica-zscore", ascending = False)


Pole,Antarctica,Arctic,Antarctica-zscore,Arctic-zscore
Date,,,,
1/29/23,-55,-17,1.508589,1.249411
1/30/23,-68,-24,0.222579,-0.046274
2/1/23,-76,-22,-0.568812,0.323921
1/31/23,-82,-32,-1.162356,-1.527058


In [29]:
dfp["Antarctica-zscore"].max()

1.5085890688299688

In [32]:
dfp["Arctic-zscore"].max()

1.2494109961334183